# Einsendeaufgabe 1: Numerische Genauigkeit und Gleichungssysteme (100 Punkte)
In dieser Aufgabe sollen Sie ein wenig mehr Erfahrung mit NumPy und numerischen Methoden gewinnen.  
Zur Erinnerung empfehle ich an dieser Stelle, die Definition der [IEEE-Fließkommazahlen](https://de.wikipedia.org/wiki/IEEE_754) zu wiederholen.  

## Addition von Zahlen (20 Punkte) 

Gegeben sei ein Array *array*, dass 100 mal die Zahl $10^{-16}$ enthält und einmal (als ersten Eintrag) die Zahl $1$. 

In [2]:
import numpy as np

array = np.concatenate(([1], np.full(100, 1e-16)))
print(array)

[1.e+00 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16]


**Aufgabe:** Addieren Sie alle Werte in `array`:
- mit der Funktion `np.sum()`
- mit einer Schleife, die von **vorne nach hinten** über `array` iteriert
- mit einer Schleife, die von **hinten nach vorne** über `array` iteriert

_Points:_ 6

In [3]:
sum_numpy = np.sum(array)
print(f"{sum_numpy=}")

sum_forward = 0.0
for x in array:
    sum_forward += x
print(f"{sum_forward=}")

sum_reverse = 0.0
for x in reversed(array):
    sum_reverse += x
print(f"{sum_reverse=}")

sum_numpy=np.float64(1.0000000000000084)
sum_forward=np.float64(1.0)
sum_reverse=np.float64(1.00000000000001)


<!-- BEGIN QUESTION -->

**Aufgabe:** Erklären Sie die Ergebnisse. Wie werden die Zahlen 1 und 1e-17 im Computer dargestellt? (ausführliche Erläuterung erforderlich!)  

_Points:_ 12

<span style="color:blue">

## Numerische Genauigkeit von verschiedenen Additionsverfahren

### Unterschiede in der Summierung

Bei der Berechnung der Summe mit unterschiedlichen Verfahren ergeben sich leicht unterschiedliche Ergebnisse:

```python
sum_numpy   = np.float64(1.0000000000000084)
sum_forward = np.float64(1.0)
sum_reverse = np.float64(1.00000000000001)
```

Diese Unterschiede entstehen durch **Rundungsfehler** in der **Gleitkommadarstellung**, die auf der begrenzten Anzahl an Bits basiert, mit der reale Zahlen intern im Computer dargestellt werden.

---

## Gleitkommazahlen im IEEE 754 Standard

Python und NumPy verwenden standardmäßig `float64` (IEEE 754 Double Precision), das wie folgt aufgebaut ist:

- **1 Bit** für das Vorzeichen  
- **11 Bits** für den Exponenten  
- **52 Bits** für die Mantisse  

Insgesamt also **64 Bit**. Die reale Zahl wird durch folgende Formel dargestellt:

```
Zahl = (-1)^s · 1.m · 2^(e - 1023)
```

Dabei ist:

- `s` das Vorzeichenbit  
- `m` die Mantisse (ohne führende 1)  
- `e` der gespeicherte Exponent mit **Bias** = 1023  

---

## Maschinengenauigkeit

Die kleinste Zahl, die zu `1.0` addiert werden kann, sodass sich das Ergebnis **numerisch unterscheidet**, nennt man **Maschinengenauigkeit** oder **machine epsilon**:

```python
np.finfo(float).eps
# Ergebnis: 2.220446049250313e-16
```

Zahlen kleiner als `eps` „verpuffen“ bei der Addition zu größeren Zahlen wie `1.0`, da sie außerhalb der darstellbaren Genauigkeit der Mantisse liegen:

```python
print(1e-16 < np.finfo(float).eps)  # True
print(1.0 + 1e-16 == 1.0)           # True
```

---

## Warum liefert `sum_forward` ein anderes Ergebnis als `sum_reverse`?

Angenommen:

```python
array = np.array([1.0] + [1e-16] * 100)
```

- `sum_forward` addiert zuerst `1.0` und dann viele kleine `1e-16`. Diese einzelnen Summanden sind kleiner als `eps` und **gehen verloren**.
- `sum_reverse` addiert zuerst die kleinen `1e-16` zu einem größeren Zwischenwert (z. B. `1e-14`), der dann zu `1.0` addiert wird und sich **merklich auswirkt**.
- `sum_numpy` verwendet oft optimierte oder paarweise Additionen (`pairwise summation`), was ebenfalls zu einem etwas besseren Ergebnis führen kann.

Gleitkommazahlen sind **nicht assoziativ**, d. h.:

```
(a + b) + c ≠ a + (b + c)
```

---

## Analyse von `1e-17` in IEEE 754

Um zu verstehen, warum `1e-17` bei der Addition zu `1.0` verloren geht, analysieren wir seine Binärdarstellung:

```python
np.frexp(1e-17)
# Ergebnis: (0.7205759403792794, -56)
```

Das bedeutet:

```
1e-17 = 0.72057594... * 2^(-56)
```

Multipliziert man Mantisse und verschiebt den Exponenten:

```
= 1.4411518807585588 · 2^(-57)
```

Zur Darstellung in IEEE 754:

- Exponent: `-57 + 1023 = 966` → Binär: `01111000110`
- Mantisse: führende `1.` ist implizit, gespeichert wird nur der Nachkommateil

Die ersten Schritte zur Berechnung der Mantissenbits:

```
0.44115188 * 2 = 0.88230376   → 0  
0.88230376 * 2 = 1.76460752   → 1  
0.76460752 * 2 = 1.52921504   → 1  
...
```

Diese 0/1-Folge wird als Bitfolge gespeichert.

---

## Bias im Exponenten

- Der Exponent wird **nicht negativ** gespeichert, sondern mit einem **Bias**.
- Für `float64` gilt:  
  ```
  Bias = 2^(11 - 1) - 1 = 1023
  ```
- Gültiger Exponentenbereich: `1` bis `2046` (Werte `0` und `2047` sind reserviert für Sonderfälle wie 0, Inf, NaN)

Beispiel:

- Mathematisch:  
  ```
  1.0 = 1.0 · 2^0 → Exponent = 0
  ```
- IEEE 754:  
  ```
  gespeicherter Exponent = 0 + 1023 = 1023 → Binär: 01111111111
  Mantisse = 0…0 (weil 1.0 keine Nachkommastellen hat)
  ```

Darstellung in 64 Bit:

```
1.0 = 0 01111111111 000...000
```

---

## Ab wann ist eine Summe nicht mehr darstellbar?

Eine einfache Faustregel:

> Zwei Zahlen `x` und `y` können exakt addiert werden, wenn  
> `| log2(x) - log2(y) | < 52` (bei `float64`)

Test in Python:

```python
def mindest_mantissenbits(x, y):
    return abs(np.log2(x) - np.log2(y))

mindest_mantissenbits(1, 1e-16) < 52  # False → nicht exakt addierbar
```

---

## Beispiel mit float128

```python
import numpy as np

x = 1
y = np.float128(1e-16)
print(x + y, np.float64(x + y))
# Ausgabe: (1.0000000000000001, 1.0)

x = 1
y = np.float128(1e-16 * 100)
print(x + y, np.float64(x + y))
# Ausgabe: (1.00000000000001, 1.00000000000001)
```

---

## Fazit

- Die Summen unterscheiden sich aufgrund der **Rundungsfehler** bei Gleitkommazahlen.
- Die **Reihenfolge der Addition beeinflusst das Ergebnis**, insbesondere wenn Zahlen mit stark unterschiedlichem Betrag summiert werden.
- **IEEE 754** und die **Maschinengenauigkeit** setzen der Genauigkeit enge Grenzen.
- Für numerisch stabile Berechnungen (z. B. große Summen, Mittelwerte) sind spezialisierte Algorithmen wie **Kahan Summation** oder **pairwise summation** zu bevorzugen.

</span>

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Welches Verfahren verwendet `np.sum()` für die Addition?

_Points:_ 2

<span style="color:blue">

<h2>Kaskadensummation</h2>

NumPy verwendet ein numerisch besseres Verfahren (partielle paarweise Summation), das in vielen Anwendungsfällen zu einer höheren Genauigkeit führt. In der numerischen Mathematik bezeichnet paarweise Summation – auch Kaskadensummation genannt – ein Verfahren zum Addieren von Gleitkommazahlen, das Rundungsfehler im Vergleich zur sequentiellen Summation deutlich reduziert.

https://numpy.org/doc/stable/reference/generated/numpy.sum.html

https://en.wikipedia.org/wiki/Pairwise_summation

</span>

<!-- END QUESTION -->

## Grenzwerte von Funktionen (20 Punkte)

Geben sei die Funktion $f(x)=\frac{e^x-1}{x}$

Es gilt: $lim_{x \to 0}f(x) = 1$

**Aufgabe:** Schreiben Sie eine Python Funktion `f(x)`, mit deren Hilfe Sie $f(x)$ für $x= 10^{-1}, 10^{-2}, \cdots , 10^{-15}$ mit NumPy berechnen und geben Sie das Ergebnis aus.

**Aufgabe:** Berechnen Sie nun die Funktion für  $x= 10^{-16}, 10^{-17}, \cdots , 10^{-20}$

_Points:_ 8

<span style="color:blue">
<h2>Berechnung der Funktion f(x) = (exp(x) - 1) / x </h2>
</span>


<span style="color:blue">
In dieser Aufgabe soll die Funktion f(x) für x = 10<sup>-1</sup>, 10<sup>-2</sup>, ..., 10<sup>-15</sup> berechnet werden. Danach soll die Funktion für noch kleinere Werte von x, nämlich x = 10<sup>-16</sup>, 10<sup>-17</sup>, ..., 10<sup>-20</sup>, berechnet werden. Der Grenzwert der Funktion für x → 0 ist 1.
</span>


In [1]:
import numpy as np

# Definition der Funktion f(x) = (e^x - 1)/x
def f(x):
        return (np.exp(x) - 1) / x

# Erzeuge die Werte x = 10^(-1), 10^(-2), ..., 10^(-15)
input_numbers = np.array([10**(-i) for i in range(1, 16)])

result_limits = f(input_numbers)

#Ausgabe
for x_val, fx in zip(input_numbers, result_limits):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-01, f(x) = 1.0517091807564771
x = 1e-02, f(x) = 1.005016708416795
x = 1e-03, f(x) = 1.0005001667083846
x = 1e-04, f(x) = 1.000050001667141
x = 1e-05, f(x) = 1.000005000006965
x = 1e-06, f(x) = 1.0000004999621837
x = 1e-07, f(x) = 1.0000000494336803
x = 1e-08, f(x) = 0.999999993922529
x = 1e-09, f(x) = 1.000000082740371
x = 1e-10, f(x) = 1.000000082740371
x = 1e-11, f(x) = 1.000000082740371
x = 1e-12, f(x) = 1.000088900582341
x = 1e-13, f(x) = 0.9992007221626409
x = 1e-14, f(x) = 0.9992007221626409
x = 1e-15, f(x) = 1.1102230246251565


In [2]:
input_numbers_2 = np.array([10**(-i) for i in range(16, 21)])
result_limits_2 = f(input_numbers_2)

#Ausgabe
for x_val, fx in zip(input_numbers_2, result_limits_2):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-16, f(x) = 0.0
x = 1e-17, f(x) = 0.0
x = 1e-18, f(x) = 0.0
x = 1e-19, f(x) = 0.0
x = 1e-20, f(x) = 0.0


<!-- BEGIN QUESTION -->

**Aufgabe:** Was fällt auf? Welche Erklärung haben Sie für das Ergebnis? (Ausführliche Erklärung erforderlich) 

_Points:_ 12

<span style="color:blue">
<h2> Erklärung des Verhaltens der Funktion bei kleinen x-Werten </h2>

Für Werte von x zwischen `1e-01` bis `1e-14` liefert die Funktion erwartungsgemäß Ergebnisse nahe bei **1**. Ab etwa 1e-15 treten jedoch drastische Änderungen auf, die durch Rundungsfehler bei der Berechnung von `e^x - 1` bedingt sind. Besonders bei sehr kleinen x-Werten wird der Unterschied zwischen `e^x` und `1` auf dem Computer nicht mehr erkennbar, was zu fehlerhaften Ergebnissen führt. Eine stabilere Berechnung kann durch die Verwendung der Funktion `np.expm1(x)` von NumPy erreicht werden.


Von `1e-01` bis `1e-14` ergibt die Funktion erwartungsgemäß Werte, die sehr nahe bei **1** liegen.  
Doch ab etwa `1e-15` ändert sich das Verhalten drastisch:

x = 1e-15, f(x) = 1.1102230246251565   
x = 1e-16, f(x) = 0.0  
x = 1e-17, f(x) = 0.0  
x = 1e-18, f(x) = 0.0  
x = 1e-19, f(x) = 0.0  
x = 1e-20, f(x) = 0.0  

<h3> Erklärung: Rundungsfehler </h3>

Für sehr kleine `x` gilt näherungsweise:  
`exp(x) ≈ 1 + x`

Wenn `x` so klein wird, dass `e^x` und `1` auf dem Computer **nicht mehr unterscheidbar** sind, wird der Ausdruck `e^x - 1` numerisch zu **null**.

> Beispiel:  
> `x = 1e-17` → `exp(x) ≈ 1.00000000000000001`  
> Die Differenz `e^x - 1` ist kleiner als die **Maschinengenauigkeit** `ε` (Epsilon).  
> Deshalb wird `e^x - 1` vom Computer als **0.0** berechnet, da ein 64-Bit-Gleitkommawert (float64) **nicht mehr ausreichend genau** ist.

<h3> Lösung: Stabilere Berechnung mit NumPy </h3>

NumPy bietet mit `np.expm1(x)` eine spezielle Funktion an, die den Ausdruck `exp(x) - 1` **numerisch stabiler** berechnet – insbesondere für sehr kleine `x`.

</span>

In [3]:
def f_stabil(x):
    return np.expm1(x) / x

input_numbers_3 = np.array([10**(-i) for i in range(10, 20)])
result_limits_3 = f_stabil(input_numbers_3)

#Ausgabe
for x_val, fx in zip(input_numbers_3, result_limits_3):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-10, f(x) = 1.00000000005
x = 1e-11, f(x) = 1.000000000005
x = 1e-12, f(x) = 1.0000000000005
x = 1e-13, f(x) = 1.00000000000005
x = 1e-14, f(x) = 1.000000000000005
x = 1e-15, f(x) = 1.0000000000000007
x = 1e-16, f(x) = 1.0
x = 1e-17, f(x) = 1.0
x = 1e-18, f(x) = 1.0
x = 1e-19, f(x) = 1.0


<!-- END QUESTION -->

## Gauß-Verfahren und LU-Zerlegung (60 Punkte) 
Obwohl man, wie wir sehen werden, lineare Gleichungssysteme einfach mit SciPy lösen kann, wollen wir in dieser Aufgabe  einmal selbst eine sogenannte LU-Zerlegung bzw. das Gauß-Verfahren implementieren. Außerdem werden wir diesen Code auch später noch verwenden, wenn wir uns mit Performance-Optimierung und Parallelisierung beschäftigen.

Das Gauß-Verfahren ist ein wichtiges Verfahren zum Lösen von linearen Gleichungssystemen, wir gehen hier schrittweise mit der Implementierung vor.

**Aufgabe:** Implementieren Sie die Funktion `b_solve` zur Lösung einer Dreiecksgleichung der Form:

$\left[ {\begin{array}{cccc} 
a_{11} & a_{12} & \cdots & a_{1n}\\ 
0 & a_{22} & \cdots & a_{2n}\\ 
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \cdots & a_{nn}\\ 
\end{array} } \right] \left[{\begin{array}{c}  
x_1\\ 
x_2\\ 
\vdots\\ 
x_n\\ 
\end{array}}\right] =  
\left[{\begin{array}{c}  
b_1\\ 
b_2\\ 
\vdots\\ 
b_n\\ 
\end{array}}\right]$ 

Hier beginnt man mit der Löung der letzten Zeile, denn es gilt:

$x_n = b_n/a_{nn} $
und dann

$x_{n-1} =  (b_{n-1}-a_{(n-1),n}*x_{n})/a_{(n-1),(n-1)}$

$x_{n-2} =  (b_{n-2}-(a_{(n-2),n}*x_{n}+a_{(n-2),(n-1)}*x_{n-1})/a_{(n-2),(n-2)}$


*Hinweis*: Achten Sie darauf, dass ihr Code nicht nur korrekt, sondern auch effizient ist. 

_Points:_ 12

<span style="color:blue">

  <h2>Backsubstitution-Verfahren</h2>

Im Folgenden wird eine Funktion zur Lösung eines linearen Gleichungssystems mit einer oberen Dreiecksmatrix implementiert. Dieses sogenannte Backsubstitution-Verfahren stellt einen zentralen Schritt im Rahmen des Gauß-Verfahrens bzw. der LU-Zerlegung dar. Dabei wird die Lösung des Systems zeilenweise von unten nach oben bestimmt. Die Implementierung legt besonderen Wert auf numerische Stabilität und Effizienz. 

Die Funktion `b_solve1` implementiert eine effiziente Rückwärtseinsetzung zur Lösung eines linearen Gleichungssystems mit oberer Dreiecksmatrix – kompakt, direkt und ohne überflüssigen Overhead.

</span>

In [3]:
import numpy as np

def b_solve1(A, b):
    """Löst ein lineares Gleichungssystem A * x = b, wobei A eine obere Dreiecksmatrix ist."""
    n = len(b)
    x = np.zeros_like(b, dtype=float)

    # Rückwärtseinsetzung
    for i in range(n - 1, -1, -1):
        summation = np.dot(A[i, i + 1:], x[i + 1:])
        x[i] = (b[i] - summation) / A[i, i]
        #print(f"x[{i}] = (b[{i}] - sum) / A[{i},{i}] = ({b[i]} - {summation}) / {A[i,i]} = {x[i]}")
    return x

# Beispiel
A = np.array([[2, 3, 5, 1],
              [0, 4, 6, 2],
              [0, 0, 7, 3],
              [0, 0, 0, 8]], dtype=float)

b = np.array([9, 8, 7, 8], dtype=float)

x = b_solve1(A, b)

# Testausgabe
print("Lösung x:", x)
print("Prüfung A @ x:", np.dot(A, x))  # Sollte gleich b sein

Lösung x: [1.60714286 0.64285714 0.57142857 1.        ]
Prüfung A @ x: [9. 8. 7. 8.]


<span style="color:blue">

Die Funktion `b_solve2` basiert auf dem Gauss-Jordan-Verfahren und löst lineare Gleichungssysteme allgemein oder im Spezialfall mit oberer Dreiecksmatrix mithilfe vektorisierter NumPy-Operationen.

</span>

In [6]:
import numpy as np

def b_solve2(A, b, upper_triangular=True):
    """Solve a system of linear equations A * x = b where A is an upper triangular matrix."""
    #kann auch nicht-triangulare Matrizen loesen 
    
    #b soll als Spaltenvektor vorliegen
    if len(b.shape) < 2:
        b = b[:, np.newaxis]
    
    #Gauss-Jordan-Algorithmus    
    Ab=np.concatenate((A,b),1)    
    for n in range(min(A.shape)-1,-1,-1):          
        if Ab[n][n]==0:
            continue
        Ab[n]=Ab[n]/Ab[n][n]  
        
        if upper_triangular:
            #Dreiecksmatrix ohne extra Schleife, 11.5 microsec
            Ab[:n] = Ab[:n] - np.outer(Ab[:n,n], Ab[n])
            
            #Dreiecksmatrix, 10 microsec
            #for m in range(n):                    
             #   Ab[m] = Ab[m] - Ab[m][n] / Ab[n][n] * Ab[n]
        else:
            #Allgemeinfall ohne extra Schleife, 20.5 microsec
            zeilen = np.arange(Ab.shape[0]) != n
            Ab[zeilen] = Ab[zeilen] - np.outer(Ab[zeilen, n], Ab[n])

            #Allgemeinfall, 14.5 microsec
            #for m in range(Ab.shape[0]):        
             #   if m != n:            
              #      Ab[m] = Ab[m] - Ab[m][n] / Ab[n][n] * Ab[n]        
            
    return Ab[:,-1:].flatten() #x steht nun in der letzten Spalte

# Beispiel
A = np.array([[2, 3, 5, 1],
              [0, 4, 6, 2],
              [0, 0, 7, 3],
              [0, 0, 0, 8]], dtype=float)

b = np.array([9, 8, 7, 8], dtype=float)

x = b_solve2(A, b)

# Testausgabe
print("Lösung x:", x)
print("Prüfung A @ x:", np.dot(A, x))  # Sollte gleich b sein

Lösung x: [1.60714286 0.64285714 0.57142857 1.        ]
Prüfung A @ x: [9. 8. 7. 8.]


**Aufgabe:** Implementieren Sie die LU-Zerlegung einer Matrix mit Pivotisierung.  
Die Funktion `lu_decomposition` soll die Matrizen L und U berechnen.  
Zu Beginn wird der Funktion eine Matrix A übergeben.  


Implementieren Sie die Funktion "in-place", d.h. die Ergebnisse für L und U werden direkt in A geschrieben.  
Dabei wird die ursprüngliche Matrix überschrieben.  
Die Funktion gibt den Pivot-Vektor zurück, der die folgende Form haben sollte:

$ 
P=\left[{\begin{array}{ccc} 
0 & 1& 0\\ 
0 & 0& 1\\ 
1 & 0& 0\\ 
\end{array} } \right] \Rightarrow \left[{\begin{array}{c}  
1\\ 
2\\ 
0\\ 
\end{array}}\right] 
$ 

D.h. es wird nur der Index des Wertes in jeder Zeile gespeichert, der nicht Null ist.

Wenn man dies macht, kann man später für die Lösung einfach den Vektor `bc = b[P]` verwenden, um auch $b$ richtig zu pivotisieren.  

Hinweise: 
- Die Pivot-Zeile lässt sich am einfachsten mit `np.argmax` finden. Die Zeile mit dem höchsten Betrag können Sie mit `np.abs` finden.

- Tauschen zweier Elemente in einem NumPy Array: `v[[a, b]] = v[[b, a]]`

- Versuchen Sie, die interne Vektorisierung von NumPy im Eliminationsschritt zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 15

<span style="color: blue;">
  <h2>LU-Zerlegung mit Pivotisierung</h2>
</span>

<span style="color: blue;">
Im folgenden Code wird die LU-Zerlegung einer Matrix A mit Pivotisierung implementiert. Die Matrix A wird in die Form A = LU zerlegt, wobei L und U die untere und obere Dreiecksmatrix sind. Zusätzlich wird ein Pivot-Vektor zurückgegeben, der die Zeilenvertauschung anzeigt. Die Berechnung erfolgt in-place und nutzt NumPy für effiziente Vektorisierung.
</span>

In [11]:
def lu_decomposition(A):
    """L und U Matrix werden innerhalb von A Matrix berechnet und der Pivot-Vektor wird ausgegeben"""
    
    n = A.shape[0]
    P = np.arange(n)

    for k in range(n):
        # Pivotisierung: Index der Zeile mit größtem Betrag in Spalte k finden
        pivot_index = np.argmax(np.abs(A[k:n, k])) + k
        if A[pivot_index, k] == 0:
            raise ValueError("Matrix ist singulaer.")

        # Zeilen tauschen in A
        if pivot_index != k:
            A[[k, pivot_index], :] = A[[pivot_index, k], :]
            P[[k, pivot_index]] = P[[pivot_index, k]]

        # Berechnung L unterhalb der Diagonale
        A[k+1:n, k] /= A[k, k]

        # Berechnung U obere Dreiecksmatrix inklusive Diagonale
        A[k+1:n, k+1:n] -= np.outer(A[k+1:n, k], A[k, k+1:n])

    return P #Pivot-Vektor

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion `custom_solve`, welche mit Hilfe von `lu_decomposition` ein lineares Gleichungssystem der Form $A*x=b$ löst.  

Hinweis: Versuchen Sie, die interne Vektorisierung von NumPy zu nutzen. Verwenden Sie dazu die Slicing-Syntax, um die innerste Schleife zu ersetzen.

_Points:_ 12

In [12]:
def custom_solve(A, b):
    """Solve a system of linear equations A * x = b and return result vector x."""
    
    #Pivot-Vektor und A (welches LU-Zerlegung enthaelt) werden berechnet
    P = lu_decomposition(A)
    
    #L und U werden aus A isoliert
    n = A.shape[0]
    L = np.tril(A, k=-1) + np.eye(n)  # untere Dreiecksmatrix mit Einsen auf der Diagonale
    U = np.triu(A)                   # obere Dreiecksmatrix inklusive Diagonale
    
    #Lc = b[P] wird geloest fuer c
    #hier wird upper_triangular=False verwendet um eine allgemeine Matrix zu loesen
    c = b_solve(L, b[P], upper_triangular=False)
    
    #Ux = c wird geloest fuer x
    x = b_solve(U, c)
    
    return x

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Testfunktion, die überprüft, ob die Lösung ihrer Implementierung des Gauß-Verfahrens richtig ist.  
Testen Sie Gleichungssysteme der Größe $16 \times 16$, $32\times 32$ und $64 \times 64$.

Sie können zum Vergleich die SciPy Funktion `linalg.solve` verwenden:

_Points:_ 6

In [27]:
import scipy.linalg as linalg

def test_custom_solve(n):    
    """vergleicht Ergebnisse von custom_solve mit linalg.solve mittels assert-Bedingung"""
    
    rng = np.random.default_rng(seed=123)
    A = rng.random((n, n))
    b = rng.random(n)
    linalg_x = linalg.solve(A, b)    
    custom_x = custom_solve(A, b)
    
    #sicherstellen, dass Ergebnisse gleich sind, float-Unpraezision ignorieren
    assert np.allclose(linalg_x, custom_x, atol=1e-6), f"Ergebnisse weichen zu stark ab bei Dimension {n}"
    
    print("Test bestanden fuer Dimension", n)
    
test_custom_solve(16)
test_custom_solve(32)
test_custom_solve(64)

Test bestanden fuer Dimension 16
Test bestanden fuer Dimension 32
Test bestanden fuer Dimension 64


<!-- END QUESTION -->

**Aufgabe:** Wir wollen nun weiter die *fertigen* SciPy Funktionen betrachten. Mit den Funktionen `lu` und `lu_factor` aus dem Packet `scipy.linalg `  können Sie eine LU-Dekomposition durchführen [[1]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu_factor.html#scipy.linalg.lu_factor) und [[2]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html).  
Beide Funktionen haben unter anderem den Parameter `overwrite_a`, der `False` oder `True` sein kann. 

Evaluieren Sie die Performance (Laufzeit) der LU-Zerlegung einer 4096x4096 Matrix, wenn Sie jeweils die Funktionen `lu_factor` und `lu` mit `overwrite_a = True` oder `overwrite_a = False` verwenden.

In der Liste `minimums` sollen die minimalen Laufzeiten in folgender Reihenfolge stehen:
1. `lu_factor` mit `overwrite_a = False`
2. `lu_factor` mit `overwrite_a = True`
3. `lu` mit `overwrite_a = False`
4. `lu` mit `overwrite_a = True`

Hinweis: Da die Matrix A bei der Verwendung von `overwrite_a = True` verändert wird, kann es zu einer Vereinfachung der Berechnung kommen, wenn die Funktion mehrfach nacheinander ausgeführt wird. Aus diesem Grund muss die `timeit` Funktion [repeat](https://docs.python.org/3/library/timeit.html#timeit.repeat) mit `repeat = 2` und `number = 1`  verwendet werden. Die Matrix wird im `setup` Parameter erstellt (aus Zufallswerten mit dem vorgegebenen "random number generator").

_Points:_ 6

In [17]:
from scipy.linalg import lu_factor, lu
from timeit import repeat

n = 4096
number = 1
repetitions = 2
# store all four minimal runtimes in this list
rng = np.random.default_rng(seed=123)
repeat_lambda = lambda x: repeat(x, setup="A = rng.random((n, n))", repeat=repetitions, number=number, globals=globals())
minimums = [min(repeat_lambda(x)) for x in ["lu_factor(A, overwrite_a=False)", "lu_factor(A, overwrite_a=True)", "lu(A, overwrite_a=False)", "lu(A, overwrite_a=True)"]]
minimums

[1.039761174999967, 1.0474625780000224, 1.4420750689999977, 1.3782401929998969]

**Aufgabe:** Untersuchen Sie den Speicherverbrauch der 4 Varianten mit memray.

Um den Einfluss der anderen Berechnungen zu minimieren, empfehlen wir hier, die Berechnungen in eine neue Datei zu schreiben und mit `!` in einer Code-Zelle memray's Command-Line Interface zu verwenden. 

Am besten tracen Sie nur die LU Funktion mit einem Context Manager!
Verwenden Sie jeweils folgende Dateinamen als `file_name` Parameter in `memray.Tracker`:

- lu_inplace.bin
- lu_notinplace.bin
- lu_factor_inplace.bin
- lu_factor_notinplace.bin

Beantworten Sie bitte die folgenden Fragen:
* Welche Version verwendet am meisten Speicher? 
* Bei welcher Version ist der Speicherverbrauch am konstantesten? 

_Points:_ 9

In [19]:
#.py files werden erstellt fuer die jeweiligen Funktionen
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    with open(file_name.replace(".bin", ".py"), "w") as file:
        #ich muss den Pfad des Environment im dem mein memray liegt hinzufuegen
           
        
        file.write(
f"""import numpy as np
from scipy.linalg import lu_factor, lu
import memray

n = 4096
rng = np.random.default_rng(seed=123)
A = rng.random((n, n))

with memray.Tracker("{file_name}", native_traces=True):
    lu{"_factor" if "_factor" in file_name else ""}(A, overwrite_a={"_inplace" in file_name})
""")

In [23]:
import memray, os

#alle .py files werden ausgefuehrt, bei mir mit python3
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    os.system(f"python3 {file_name.replace(".bin", ".py")}")

ModuleNotFoundError: No module named 'memray'

In [24]:
#Speichernutzung wird aus allen .bin files gelesen
for file_name in ["lu_inplace.bin", "lu_notinplace.bin", "lu_factor_inplace.bin", "lu_factor_notinplace.bin"]:
    reader = memray.FileReader(file_name)
    mem_use = 0

    for allocation in reader.get_allocation_records():
        mem_use += allocation.size

    print(f"Speichernutzung von {file_name[:-4]} : {round(mem_use / 2**20)}MB") 

NameError: name 'memray' is not defined

Die Funktion lu_factor verwendet konstant nur 144MB an Speicher.

Funktion lu verwendet deutlich mehr Speicher mit 400MB "inplace" und sogar 528MB "notinplace"


In [ ]:
#!rm -rf lu_* memray-*